<a href="https://colab.research.google.com/github/dionysus98/l_torch/blob/main/01_torch_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torch workflow

- Resources:
  - [Pytorch Book](https://www.learnpytorch.io/)
  - [Source code for the book](https://github.com/mrdbourke/pytorch-deep-learning)

In [1]:
to_cover = {
    1: "data (prepare and load)",
    2: "build a model",
    3: "fitting(traning) the modal",
    4: "making predictions and evaluating the model",
    5: "saving, loading the model",
    6: "putting it all together"
    }

In [2]:
import torch
from torch import nn ## neural networks
import matplotlib.pyplot as plt

torch.__version__

'2.3.0+cu121'

## 1. Data - Preparing and Loading